In [19]:
import json
import tensorflow as tf
import pandas as pd

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [20]:
vocab_size = 5000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 1400

In [21]:
text = pd.read_csv("youtube-comments.csv")

In [22]:
sentences = list(text["content"])
labels = list(text["class"])

In [23]:
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

In [24]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok, filters = "!\\w*")
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [25]:
len(word_index)

4542

In [26]:
# Need this block to get it to work with TensorFlow 2.x
import numpy as np
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [27]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 16)           48000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 24)                408       
                                                                 
 dense_1 (Dense)             (None, 1)                 25        
                                                                 
Total params: 48,433
Trainable params: 48,433
Non-trainable params: 0
_________________________________________________________________


In [29]:
num_epochs = 30
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=2)

Epoch 1/30
44/44 - 2s - loss: 0.6925 - accuracy: 0.5221 - val_loss: 0.6889 - val_accuracy: 0.4928 - 2s/epoch - 42ms/step
Epoch 2/30
44/44 - 0s - loss: 0.6875 - accuracy: 0.5414 - val_loss: 0.6770 - val_accuracy: 0.4928 - 181ms/epoch - 4ms/step
Epoch 3/30
44/44 - 0s - loss: 0.6782 - accuracy: 0.5879 - val_loss: 0.6527 - val_accuracy: 0.5270 - 167ms/epoch - 4ms/step
Epoch 4/30
44/44 - 0s - loss: 0.6589 - accuracy: 0.7664 - val_loss: 0.6114 - val_accuracy: 0.9101 - 180ms/epoch - 4ms/step
Epoch 5/30
44/44 - 0s - loss: 0.6251 - accuracy: 0.8414 - val_loss: 0.5657 - val_accuracy: 0.8867 - 167ms/epoch - 4ms/step
Epoch 6/30
44/44 - 0s - loss: 0.5756 - accuracy: 0.8443 - val_loss: 0.5036 - val_accuracy: 0.9317 - 159ms/epoch - 4ms/step
Epoch 7/30
44/44 - 0s - loss: 0.5126 - accuracy: 0.8857 - val_loss: 0.4391 - val_accuracy: 0.9424 - 164ms/epoch - 4ms/step
Epoch 8/30
44/44 - 0s - loss: 0.4461 - accuracy: 0.9029 - val_loss: 0.3741 - val_accuracy: 0.9245 - 184ms/epoch - 4ms/step
Epoch 9/30
44/44 -

In [32]:
sentence = ["follow link and win money", "video looks cool, mighs save for later"]
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(model.predict(padded))

1/1 [==============================] - 0s 26ms/step
[[0.7864406 ]
 [0.22341347]]
